In [ ]:
import configparser, os
import numpy as np
import openai
import pandas as pd
import pickle
import tiktoken

# Set the API key and load the configuration file
config = configparser.ConfigParser()
config.read('../config.ini')
openai.api_key = config['openai']['api_key']
os.environ['OPENAI_API_KEY'] = config['openai']['api_key']

# Model Fine-Tuning
In this notebook we will retrain the model by fine-tuning GPT-3 with an example dataset.
We can then ask questions to the model related to the dataset we just fine-tuned it with.

![Model Selection](../images/fine_tuning_model_selection.png)


*Illustrative examples of text classification performance on the Stanford Natural Language Inference (SNLI) Corpus, in which ordered pairs of sentences are classified by their logical relationship: either contradicted, entailed (implied), or neutral. Default fine-tuning parameters were used when not otherwise specified.*

For complex tasks, requiring subtle interpretation or reasoning or prior knowledge or coding ability, the performance gaps between models can be larger, and better models like curie or text-davinci-002 could be the best fit.

**A single project might end up trying all models. One illustrative development path might look like this:**
- Test code using the cheapest & fastest model (ada)
- Run a few early experiments to check whether your dataset works as expected with a middling model (curie)
- Run a few more experiments with the best model to see how far you can push performance (text-davinci-002)
- Once you have good results, do a training run with all models to map out the price-performance frontier and select the model that makes the most sense for your use case  (ada, babbage, curie, davinci, text-davinci-002)

**Another possible development path that uses multiple models could be:**
- Starting with a small dataset, train the best possible model (text-davinci-002)
- Use this fine-tuned model to generate many more labels and expand your dataset by multiples
- Use this new dataset to train a cheaper model (ada)


More info about model fine-tuning can be found [here](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#).

In this notebook we will fine-tune the model with FAQ data coming from https://coolblue.be. We will then ask questions to the model related to the dataset we just fine-tuned it with.

## Step 1: Preprocess the FAQ data
The preprocessing step is crucial for ensuring that the FAQ data is in a format that can be easily processed by the model. Here are some common preprocessing steps for text data:

1. **Lowercase the text**: Converting all text to lowercase can help reduce the size of the vocabulary and make the model more robust to variations in capitalization.

2. **Tokenize the text**: Tokenization involves splitting the text into individual tokens (e.g., words or subwords) that can be fed into the model. This can be done using a tokenizer, such as the one provided by the Hugging Face Transformers library.

3. **Remove punctuation and special characters**: Removing punctuation and special characters can help reduce the size of the vocabulary and make the model more robust to variations in the input.

4. **Remove stop words**: Stop words are common words that are unlikely to carry much meaning, such as "a", "an", "the", etc. Removing stop words can reduce the size of the vocabulary and improve the efficiency of the model.

5. **Convert words to IDs**: The model operates on numbers, not words, so we need to convert each word in the text to a unique integer ID. This can be done using a vocabulary, which maps each word to an ID.

In [ ]:
COMPLETIONS_MODEL = "text-davinci-003"
EMBEDDING_MODEL = "text-embedding-ada-002"

In [ ]:
prompt = """Answer the question only when the answer is in the provided context, and if you're unsure of the answer, say "I can't help you with this, please contact customer support at support@coolblue.be".

Context:
You can pay online with Bancontact (card and app), credit card (Visa or MasterCard), PayPal, Apple Pay, Coolblue gift cards, or a bank transfer.
In the store, you can easily pay with Bancontact, credit card, cash, Apple Pay, Coolblue gift cards, Consumption Passes, and EcoCheques.

Q: WWhat kind of payment methods are supported ?
A:"""


# Create a function to call the OpenAI API using the COMPLETIONS_MODEL



In [ ]:
# Load the coolblue_faq.csv file and set the columns category and question as an index


# Create a column named tokens and harcode the value to 50 
df['tokens'] = 50

print(f"{len(df)} rows in the data.")
df.sample(5)

In [ ]:
def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
      model=model,
      input=text
    )
    return result["data"][0]["embedding"]

def compute_doc_embeddings(df: pd.DataFrame) -> dict[tuple[str, str], list[float]]:
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    
    Return a dictionary that maps between each embedding vector and the index of the row that it corresponds to.
    """
    return {
        idx: get_embedding(r.context) for idx, r in df.iterrows()
    }

In [ ]:
def load_embeddings(fname: str) -> dict[tuple[str, str], list[float]]:
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.category, r.question): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

In [ ]:
document_embeddings = compute_doc_embeddings(df)

In [ ]:
# An example embedding:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

So we have split our document library into sections, and encoded them by creating embedding vectors that represent each chunk. Next we will use these embeddings to answer our users' questions.

# 2) Find the most similar document embeddings to the question embedding

At the time of question-answering, to answer the user's query we compute the query embedding of the question and use it to find the most similar document sections. Since this is a small example, we store and search the embeddings locally. If you have a larger dataset, consider using a vector search engine like [Pinecone](https://www.pinecone.io/) or [Weaviate](https://github.com/semi-technologies/weaviate) to power the search.

In [ ]:
def vector_similarity(x: list[float], y: list[float]) -> float:
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query: str, contexts: dict[(str, str), np.array]) -> list[(float, (str, str))]:
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [ ]:
order_document_sections_by_query_similarity("What payment methods are available?", document_embeddings)[:5]

In [ ]:
order_document_sections_by_query_similarity("What types of warranty are there ?", document_embeddings)[:5]

# 3) Add the most relevant document sections to the query prompt

Once we've calculated the most relevant pieces of context, we construct a prompt by simply prepending them to the supplied query. It is helpful to use a query separator to help the model distinguish between separate pieces of text.

In [ ]:
MAX_SECTION_LEN = 500
SEPARATOR = "\n* "
ENCODING = "gpt2"  # encoding for text-davinci-003

encoding = tiktoken.get_encoding(ENCODING)
separator_len = len(encoding.encode(SEPARATOR))

f"Context separator contains {separator_len} tokens"

In [ ]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.context.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [ ]:
prompt = construct_prompt(
    "Do I get an invoice with my order ?",
    document_embeddings,
    df
)

print("===\n", prompt)

## Answer the user's question based on the context.
Now that we've retrieved the relevant context and constructed our prompt, we can finally use the Completions API to answer the user's query.

In [ ]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL,
}

In [ ]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embeddings: dict[(str, str), np.array],
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = openai.Completion.create(
                prompt=prompt,
                **COMPLETIONS_API_PARAMS
            )

    return response["choices"][0]["text"].strip(" \n")

In [ ]:
answer_query_with_context("What are the accepted payment methods", df, document_embeddings)


By combining the Embeddings and Completions APIs, we have created a question-answering model which can answer questions using a large base of additional knowledge. It also understands when it doesn't know the answer!

For this example we have used a dataset of Wikipedia articles, but that dataset could be replaced with books, articles, documentation, service manuals, or much much more. **We can't wait to see what you create with GPT-3!**